In [2]:
from asyncore import read
import spacy
from spacy import displacy
#from utils.utils import count_occurrences, get_entities, search_with_google, wiki_content
import json

nlp = spacy.load('it_core_news_sm')

with open(f'assets/test_sentences.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

FileNotFoundError: [Errno 2] No such file or directory: 'assets/test_sentences.txt'

In [2]:
sentence_list = list(doc.sents) 

# Opening JSON file
f = open('./assets/italian_cities.json')
italian_cities = json.load(f)
f.close()
# Count occurrences
counter = count_occurrences(doc, italian_cities, "name")  

# Max occurrence
context = max(counter, key=counter.get) 
print(context)

# Get entities without duplicates
searchable_entities = get_entities(doc, counter)

print(searchable_entities)
#print(searchable_entities)

# Search addresses with Google
search_with_google(searchable_entities, context)

Torino
['Novecento', 'Italia', 'Ottocento', 'Novecento.', 'Impresa Porcheddu', 'Esposizione generale italiana', 'Borgo Medievale', 'Vetreria Albano&Macario', 'Terrazza Solferino', 'Mobilificio Torinese F. Cesare Gandolfo', 'Albergo Rocciamelone di', 'Francia', 'Villino Raby', 'Casa Fenoglio-Lafleur', 'Casa Rossi-Galateri', 'Passalacqua', 'Casa Girardi', 'Cibrario 54.', 'Crocetta', 'Casa Maffei']
https://google.it/search?q=Novecento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Italia+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Ottocento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Novecento.+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Impresa+Porcheddu+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Esposizione+generale+italiana+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Borgo+Medievale+Torino&hl=it
Address:  None
Activity:  None
ht

In [3]:
print(spacy.explain("LOC"))
print(spacy.explain("ADP"))
print(spacy.explain("flat"))
print(spacy.ex)

Non-GPE locations, mountain ranges, bodies of water
adposition
flat multiword expression


In [26]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="PoI_geocoder")

location = geolocator.geocode("Chiesa della Salute", exactly_one = False)

print("RESULTS: ", location)
for loc in location: 
    print(loc.address)
    #Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
    print((loc.latitude, loc.longitude))
    print(loc.point)
    #(40.7410861, -73.9896297241625)
    print(loc.raw)
    #{'place_id': '9167009604', 'type': 'attraction', ...}


import geocoder
g = geocoder.google('Torino')
print(g.latlng)


import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
results = r.json()['results']
print(results)
#location = results[0]['geometry']['location']
#print(location['lat'], location['lng'])

RESULTS:  [Location(Chiesa Della Salute, Via Breglio, Borgo Vittoria, Circoscrizione 5, Torino, Piemonte, 10147, Italia, (45.0989972, 7.6796077, 0.0))]
Chiesa Della Salute, Via Breglio, Borgo Vittoria, Circoscrizione 5, Torino, Piemonte, 10147, Italia
(45.0989972, 7.6796077)
45 5m 56.3899s N, 7 40m 46.5877s E
{'place_id': 12472809, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1262193690, 'boundingbox': ['45.0989472', '45.0990472', '7.6795577', '7.6796577'], 'lat': '45.0989972', 'lon': '7.6796077', 'display_name': 'Chiesa Della Salute, Via Breglio, Borgo Vittoria, Circoscrizione 5, Torino, Piemonte, 10147, Italia', 'class': 'highway', 'type': 'bus_stop', 'importance': 0.30010000000000003, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/transport_bus_stop2.p.20.png'}
None
[]


In [6]:
import spacy
from spacy import displacy
from spacy.tokens import Doc

nlp = spacy.load('it_core_news_sm', exclude=["ner"])
ner_ita = spacy.load('it_nerIta_trf')

print("nomi: ", nlp.pipe_names)
# give this component a copy of its own tok2vec
#ner_ita.replace_listeners("tok2vec", "ner", ["model.tok2vec"])

# now you can put the drug component before or after the other ner
# This will print a W113 warning but it's safe to ignore here
nlp.add_pipe("transformer", name="trf_ita", source=ner_ita, last=True)

nlp.add_pipe("ner", name="ner_ita", source=ner_ita, last=True)

print("nomi: ", nlp.pipe_names)

sentence = "Fra questi il Teatro Balbo e il Teatro di Torino, distrutti durante un bombardamento nel 1943, il Teatro Rossini, bruciato in un incendio nel 1941, i teatri Adua, Gianduja, Gerbino e Arena torinese, smantellati e, infine, il Teatro Nazionale riconvertito a Cinema"
#sentence = "All'interno del perimetro cittadino figurano cinque siti iscritti nella lista del Patrimonio Mondiale dell'UNESCO: Palazzo Reale e la zona di comando (o musei reali); Palazzo Madama; Palazzo Carignano; Castello del Valentino; Villa della Regina"                        
#sentence = "Inoltre, i comuni di cintura dell'area urbana e metropolitana cittadina possiedono un sistema bibliotecario integrato denominato SBAM (Sistema Bibliotecario Area Metropolitana) composto di 65 biblioteche per un totale di circa 1.700.000 documenti.\
#L'Archivio di Stato di Torino custodisce i documenti della corte e dell'amministrazione sabauda, sin dall'epoca medievale."
#sentence = "Attorno alla città, ad anello, vi sono il Parco della Mandria e il Parco della Palazzina di caccia di Stupinigi, antiche riserve di caccia dei Savoia, e quelli situati sulla collina torinese quali il Parco della Rimembranza o il Parco Europa."
#sentence = " Fra i teatri maggiori, con una capienza superiore a 400 posti, vi sono il Teatro Regio, in cui si tenne la prima de La bohème di Puccini, il Teatro Carignano, il Teatro Alfieri e il Teatro Colosseo; importante, poi, è il Teatro Gobetti, sede principale del sopracitato teatro stabile cittadino."
#sentence = "Torino sorge nella pianura delimitata dai fiumi Stura di Lanzo, Sangone e Po (quest'ultimo attraversa la città da sud verso nord), di fronte allo sbocco di alcune vallate alpine: Val di Susa, che collega la città con la vicina Francia attraverso il Traforo del Frejus, Valli di Lanzo, Val Sangone. "
#sentence = "Le collezioni di arte antica, la cui raccolta fu iniziata dal duca Emanuele Filiberto di Savoia nella seconda metà del Cinquecento, sono conservate nel Museo di antichità, che raccoglie anche le principali testimonianze archeologiche piemontesi dal Paleolitico al Tardo Medioevo."
#sentence = "A Torino hanno inoltre sede la Fondazione europea per la formazione professionale (ETF), una delle agenzie dell'Unione europea, e l'Autorità di regolazione dei trasporti (ART), autorità indipendente italiana."
#sentence = "A causa delle imponenti opere di pianificazione urbana effettuate a partire dal XVI secolo da parte della corte sabauda, Torino ha conservato pochi monumenti appartenenti all'epoca medioevale e rinascimentale. Tra questi si possono annoverare: Palazzo Madama e Casaforte degli Acaja, il cui corpo centrale fu costruito fra i secoli XIII e XV in stile gotico ampliando e inglobando la struttura dell'originale Porta Decumana d'epoca romana e conferendo al palazzo l'aspetto di un castello, che verrà completato nei secoli successivi con la facciata settecentesca dello Juvarra; Chiesa di San Domenico, situata nell'omonima via; fu edificata nel XIV secolo e rappresenta l'unico monumento completo autenticamente medioevale dell'intera città, pur se modificata in epoca barocca e riportata successivamente alle originali forme gotiche tramite restauri conservativi ottocenteschi; Duomo; edificato fra il 1491 e il 1498, costituisce l'unico esempio di luogo di culto in stile rinascimentale della città; campanili romanici della basilica della Consolata (risalente al X secolo), del Duomo (quattrocentesco, poi completato nel XVIII secolo dallo Juvarra) e della chiesa di Sant'Agostino (XV secolo); Casa dei Romagnano, resti medievali in via dei Mercanti 9; Casa del Senato, in piazza IV Marzo 17; Casa del Pingone, in via IV Marzo; casa con torre medievale (mascherata), già residenza di Filiberto Pingone, storico che nel 1577 scrisse la prima storia di Torino, intitolata Augusta Taurinorum; Casa Broglia; Palazzo Scaglia di Verrua (XV secolo); Mastio della Cittadella, unico edificio superstite del complesso sistema difensivo torinese, edificato a partire dal 1564 in uno stile a cavallo fra quello rinascimentale e quello barocco."
#sentence = "L'ammontare delle precipitazioni annue, 833 mm, si è conservata sostanzialmente immutata dalla metà dell'Ottocento a oggi. "
#sentence = "La rete di trasporti di Torino è gestita dal GTT, acronimo di Gruppo Torinese Trasporti , nato nel 2003 dalla fusione di ATM e SATTI ."
#sentence = "Altre sale teatrali minori comprendono l'Alfa Teatro"
#sentence = "Erano inoltre presenti in Torino i cimiteri, ora non più in uso: # - Cimitero Monumentale (già Cimitero Generale): sito in Corso Novara, 135 (quartiere Regio Parco), è il primo cimitero di Torino per dimensioni; contiene anche un Tempio Crematorio e un Cimitero giudaico; vi sono numerosi monumenti funebri di personaggi famosi e di pregevole fattura;\
# - il Cimitero Parco (detto anche \"cimitero del Gerbido\"): costruito nel 1972 nella zona all'estremo sud di Torino, è il secondo cimitero di Torino per dimensioni;\
# - un Cimitero di Mirafiori: piccolo cimitero sito in corso Unione Sovietica 650, nella zona Sud di Torino;\
# - il Cimitero di Sassi: sito nell'omonima strada al n. 24;\
# - il Cimitero di Cavoretto: sito nell'omonimo borgo collinare;\
# - il Cimitero di Abbadia di Stura: sito in Strada di Settimo 307, nella zona nord. #"
#sentence = "Un colpo all'italiana . Un gruppo di ladri inglesi arriva a Torino a bordo di tre Mini Cooper per organizzare una rapina ai danni di un convoglio che trasporta i ricavi della FIAT dall' Aeroporto di Torino-Caselle fino alla città."
#sentence = "Nel suo territorio sono inoltre presenti aree ed edifici inclusi in due beni protetti dall' UNESCO :\
#     alcuni palazzi e zone facenti parte del circuito di residenze sabaude in Piemonte ( patrimonio dell'umanità ) \
#        e l'area delle colline del Po ( riserva della biosfera ). Città dalla storia bimillenaria, fu fondata probabilmente\
#             nei pressi della posizione attuale, attorno al III secolo a.C. , dai Taurini , quindi trasformata in \
#                colonia romana da Augusto col nome di Iulia Augusta Taurinorum nel I secolo a.C. . \
#                    Dopo il dominio ostrogoto , fu capitale di un importante ducato longobardo , \
#                        per poi passare, dopo essere divenuta capitale di marca carolingia , \
#                            sotto la signoria nominale dei Savoia nell' XI secolo . \
#                                Città dell' omonimo ducato , nel 1563 ne divenne capitale."
#sentence = "Abitanti censiti (migliaia) Al  1º gennaio 2021 risiedevano a Torino 128 238 stranieri, costituenti il 14,9% della popolazione totale. Di seguito sono riportati i gruppi più consistenti : # ^ Romania , 44 383 ; ^ Marocco , 15 427 ; ^ Cina , 8 195 ; ^ Perù , 7 120 ; ^ Nigeria , 5 532 ; ^ Egitto , 5 405 ; ^ Albania , 5 061 ; ^ Filippine ,3 708 ; ^ Moldavia , 3 069 ; ^ Bangladesh , 2 120 ."
#sentence = "Vi sono quattro musei nazionali (Museo del cinema, Museo dell'automobile, Museo della montagna, Museo del Risorgimento) e numerosi altri musei di rilevanza nazionale ed internazionale come il Museo egizio, l'Armeria Reale, ora ricompresa nei Musei Reali, il Museo d'Arte Orientale, il Museo dell'Astronomia e Planetario, il J-Museum a cui si aggiungeva, fino al 2015, il Museo dello sport. "
#sentence = " Da ricordare il Museo dell'astronomia e Planetario di Torino , che sorge accanto dell'Osservatorio astronomico di Torino a Pino Torinese , cittadina collinare nei pressi immediati del capoluogo piemontese. "
#sentence = "via sibilla aleramo 1 ,torino."
doc_ner = nlp(sentence)

#displacy.render(doc, style="dep", jupyter=True)

displacy.render(doc_ner, style="ent", jupyter=True)
displacy.render(doc_ner, style="dep", jupyter=True)

entities = doc_ner.ents
for entity in entities:
    print(entity.text, entity.label_)

for token in doc_ner:
    print(token.text, token.pos_)
#sentences = list(doc.sents)

#for sentence in sentences:
#    print("Frasi", sentence.text)

#sentence = sentences[0]

#doc = nlp(sentence)
#for token in doc:
#    print(token.text, token.pos_, token.dep_)
#displacy.render(doc, style="ent", jupyter=True)
#displacy.render(doc, style="dep", jupyter=True)
                

nomi:  ['tok2vec', 'morphologizer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']
nomi:  ['tok2vec', 'morphologizer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'trf_ita', 'ner_ita']


il Teatro Balbo FAC
il Teatro di Torino FAC
1943 DATE
il Teatro Rossini FAC
1941 DATE
Adua FAC
Gianduja FAC
Gerbino FAC
Arena torinese FAC
il Teatro Nazionale FAC
Cinema FAC
Fra ADP
questi PRON
il DET
Teatro NOUN
Balbo PROPN
e CCONJ
il DET
Teatro NOUN
di ADP
Torino PROPN
, PUNCT
distrutti VERB
durante ADP
un DET
bombardamento NOUN
nel ADP
1943 NUM
, PUNCT
il DET
Teatro NOUN
Rossini PROPN
, PUNCT
bruciato ADJ
in ADP
un DET
incendio NOUN
nel ADP
1941 NUM
, PUNCT
i DET
teatri NOUN
Adua PROPN
, PUNCT
Gianduja PROPN
, PUNCT
Gerbino PROPN
e CCONJ
Arena PROPN
torinese ADJ
, PUNCT
smantellati VERB
e CCONJ
, PUNCT
infine ADV
, PUNCT
il DET
Teatro NOUN
Nazionale ADJ
riconvertito ADJ
a ADP
Cinema NOUN


In [21]:
import wikipedia
wikipedia.set_lang("it")

#wikipedia.summary("Torino")
try:
    to = wikipedia.page("Chiesa della Misericordia (Torino)", preload=False)
except wikipedia.exceptions.DisambiguationError as e:
    print(e.options)
    print(e.options[0])
    to = wikipedia.page(e.options[0], preload=False)
    print(to.summary)
    #to = wikipedia.page(e.options[0], preload=False)
#wikipedia.categories("Torino")
#to.categories
print(to.summary)

La chiesa della Misericordia è una delle chiese di Torino, sita in via Barbaroux 41. È dedicata a san Giovanni Battista (era detta anche "chiesa di San Giovanni Decollato"). Deve il nome alla Arciconfraternita della Misericordia, che nel XVIII secolo ne prese possesso e la ristrutturò, così nominata poiché i suoi membri avevano il compito di confortare i condannati a morte, accompagnarli al patibolo, curarne le successive esequie e far celebrare messe in suffragio delle loro anime.
